In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from birdCLEFloaddata import load_audiofile,load_metadata,get_melspectrogram
from birdCLEFFunctions import Dynamic_CNN
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import Resize
from torch.utils.data import DataLoader, TensorDataset

plt.rcParams['figure.dpi'] = 600


In [25]:
path = r"C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCLEFproject\data\birdCLEF2023"

meta_data = load_metadata(path)
meta_data.head(10)


,latitude,longitude,common_name,rating,filename
0,4.3906,38.2788,African Bare-eyed Thrush,4.0,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
1,-2.9524,38.2921,African Bare-eyed Thrush,3.5,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
2,-2.9524,38.2921,African Bare-eyed Thrush,3.5,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
3,-2.9524,38.2921,African Bare-eyed Thrush,5.0,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
4,-2.9524,38.2921,African Bare-eyed Thrush,4.5,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
5,-2.9524,38.2921,African Bare-eyed Thrush,3.5,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
6,-2.9965,37.6244,African Bare-eyed Thrush,3.0,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
7,-4.0904,37.8807,African Bare-eyed Thrush,5.0,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
8,-4.0904,37.8807,African Bare-eyed Thrush,5.0,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
9,4.8403,38.6988,African Bare-eyed Thrush,4.0,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...


In [4]:
#Generate audio data
audio_data = []
for i in range(50):
    audio, sr = load_audiofile(meta_data['filename'][i])
    audio_data.append(audio)

In [29]:
#Generate spectrograms
spectrograms = []
for i in range(1000):
    spectrograms.append(get_melspectrogram(meta_data['filename'][i]))



In [41]:
shape = []
for j,i in enumerate(spectrograms):
    shape.append(np.shape(i)[1])


In [44]:
np.sort(shape)

array([   71,    74,    85,   129,   131,   134,   144,   157,   161,
         172,   200,   211,   214,   216,   216,   237,   238,   242,
         245,   245,   245,   247,   249,   253,   254,   254,   258,
         262,   263,   267,   278,   283,   284,   298,   301,   304,
         305,   309,   311,   316,   317,   319,   319,   324,   334,
         343,   346,   347,   347,   347,   347,   348,   349,   349,
         356,   358,   373,   376,   376,   376,   376,   377,   378,
         379,   386,   387,   391,   394,   396,   397,   407,   414,
         420,   422,   425,   427,   428,   431,   431,   435,   436,
         437,   438,   438,   440,   441,   443,   443,   443,   445,
         446,   448,   449,   449,   449,   451,   451,   454,   454,
         463,   469,   471,   477,   479,   482,   484,   485,   489,
         490,   494,   495,   495,   495,   498,   499,   500,   500,
         503,   503,   508,   512,   512,   512,   520,   521,   526,
         529,   536,

In [27]:
#Neural network code (Work in progress)
unique_common_names = meta_data['common_name'].unique().tolist()

input_dim = 1  
output_dim = len(unique_common_names)
layers = [(16, 3), (32, 3)]

train_ratio = 0.8
train_size = int(train_ratio * len(spectrograms))
train_data = spectrograms[:train_size]
val_data = spectrograms[train_size:]

# Extract the labels from the meta_data
train_labels = meta_data['common_name'].tolist()[:train_size]
val_labels = meta_data['common_name'].tolist()[train_size:]

# Create a dictionary to map the unique labels to integer values, so we can use them as outputs
label_mapping = {label: index for index, label in enumerate(set(train_labels))}

# Convert the labels to their corresponding integer values
train_labels = [label_mapping.get(label, -1) for label in train_labels]
val_labels = [label_mapping.get(label, -1) for label in val_labels]

# Remove data points with missing labels from the validation set
val_data = []
val_labels_filtered = []
for label, data_point in zip(val_labels, val_data):
    if label != -1:
        val_labels_filtered.append(label)
        val_data.append(data_point)
val_labels = val_labels_filtered

# Convert the label lists to PyTorch tensors
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)


# Resize spectrograms to a common size
resize = Resize((128, 128))
spectrograms_resized = [resize(spec.transpose((2, 0, 1))) for spec in spectrograms[:train_size]]

# Convert spectrograms to a PyTorch Tensor
spectrograms_tensor = torch.stack([torch.Tensor(spec) for spec in spectrograms_resized])

# Create DataLoader for training and validation sets
batch_size = 32
train_dataset = TensorDataset(spectrograms_tensor, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(torch.stack(val_data), val_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


model = Dynamic_CNN(input_dim, output_dim, layers)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)




TypeError: transpose() received an invalid combination of arguments - got (tuple), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [28]:
np.shape(spectrograms)

c:\Users\zhakk\anaconda3\envs\AML\lib\site-packages\numpy\core\fromnumeric.py:2007: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  result = asarray(a).shape
c:\Users\zhakk\anaconda3\envs\AML\lib\site-packages\numpy\core\fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(1000,)